In [1]:
import pickle
import numpy as np
import pandas as pd
import math
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from df_selector import *
from outliener import Outlier
from visualization import *

In [3]:
user_engagement_df = pd.read_csv("../data/user_engagement_data.csv")
user_engagement_df.head()

,MSISDN/Number,cluster,xDR Sessions,Dur. (ms),Total Data Volume (Bytes)
0,3.362632e+10,0,18.0,8791927.0,7.971167e+09
1,3.361489e+10,0,17.0,9966898.0,8.846226e+09
2,3.362578e+10,0,17.0,15466706.5,8.499621e+09
3,3.365973e+10,2,16.0,4035428.0,7.705863e+09
4,3.367588e+10,2,15.0,4865947.0,7.891111e+09


### Engagement Score

In [6]:
eng_df = user_engagement_df.set_index('MSISDN/Number')[
    ['xDR Sessions', 'Dur. (ms)', 'Total Data Volume (Bytes)']]

In [7]:
# scale data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(eng_df)
scaled_data

array([[ 1.50467048,  0.32030713,  0.59916799],
       [ 1.04870973,  0.67640567,  1.21996351],
       [ 1.04870973,  2.34323301,  0.97407009],
       [ 0.59274898, -1.12124534,  0.41095235],
       [ 0.13678823, -0.86953989,  0.54237319],
       [ 0.13678823,  0.46805591,  0.98482029],
       [-0.77513328,  0.3060589 , -1.06996596],
       [-1.23109403, -0.43863063, -1.52556286],
       [-1.23109403, -0.7658769 , -1.16257441],
       [-1.23109403, -0.91876787, -0.97324419]])

In [8]:
normalized_data = normalize(scaled_data)
normalized_data

array([[ 0.91139738,  0.19401396,  0.3629234 ],
       [ 0.60092032,  0.38758667,  0.69905031],
       [ 0.38193413,  0.85339217,  0.3547508 ],
       [ 0.44460594, -0.84101762,  0.30824491],
       [ 0.13230129, -0.84101716,  0.52458222],
       [ 0.12447343,  0.4259177 ,  0.89615873],
       [-0.57153978,  0.22567066, -0.78893285],
       [-0.61284638, -0.21835309, -0.75943482],
       [-0.66243986, -0.41211099, -0.62557011],
       [-0.67698438, -0.50523476, -0.53519154]])

In [14]:
with open("../models/user_engagement.pkl", "rb") as f:
    kmeans1 = pickle.load(f)

In [15]:
less_engaged_cluster = 3

In [16]:
distance = kmeans1.fit_transform(normalized_data)
distance_from_less_engaged_cluster = list(
    map(lambda x: x[less_engaged_cluster], distance))
user_engagement_df['engagement_score'] = distance_from_less_engaged_cluster
user_engagement_df.head()

IndexError: index 3 is out of bounds for axis 0 with size 3

#### Experience Score

In [17]:
exp_df = user_experience_df.set_index('MSISDN/Number')[
    ['Total Avg RTT (ms)', 'Total Avg Bearer TP (kbps)', 'Total TCP Retrans. Vol (Bytes)']]

In [18]:
# scale data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(exp_df)
scaled_data

array([[-0.17015638, -0.49427243, -0.10860156],
       [-0.19190879, -0.49373249, -0.07950754],
       [-0.15130428, -0.49377944, -0.10287579],
       ...,
       [-0.16145541, -0.25273304, -0.12081633],
       [-0.17885734, -0.49333341, -0.12067974],
       [-0.14985412, -0.49528188, -0.12126092]])

In [19]:
normalized_data = normalize(scaled_data)
normalized_data

array([[-0.31870262, -0.92577146, -0.20341054],
       [-0.35827194, -0.92174253, -0.14843155],
       [-0.28732972, -0.93769659, -0.19536309],
       ...,
       [-0.49936101, -0.78167108, -0.3736695 ],
       [-0.33216904, -0.91620551, -0.22412316],
       [-0.28195947, -0.93190237, -0.22815965]])

In [20]:
with open("../models/user_experience.pkl", "rb") as f:
    kmeans2 = pickle.load(f)

In [21]:

worst_experience_cluster = 1

In [22]:
distance = kmeans2.fit_transform(normalized_data)
distance_from_worst_experience_cluster = list(
    map(lambda x: x[worst_experience_cluster], distance))
user_experience_df['experience_score'] = distance_from_worst_experience_cluster
user_experience_df.head()

,MSISDN/Number,cluster,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),experience_score
0,3.360100e+10,1,46.0,76.0,2895381.0,0.046970
1,3.360100e+10,1,31.0,99.0,9272453.0,0.100926
2,3.360100e+10,1,59.0,97.0,4150403.0,0.053453
3,3.360101e+10,1,84.0,248.0,2396.0,0.092104
4,3.360101e+10,2,119.0,56844.0,9738882.0,1.899030


### Satisfaction Score

In [23]:
user_engagement_df.rename(columns={'cluster': 'engagement_cluster'}, inplace=True)
user_engagement_df.head()

,MSISDN/Number,engagement_cluster,xDR Sessions,Dur. (ms),Total Data Volume (Bytes)
0,3.362632e+10,0,18.0,8791927.0,7.971167e+09
1,3.361489e+10,0,17.0,9966898.0,8.846226e+09
2,3.362578e+10,0,17.0,15466706.5,8.499621e+09
3,3.365973e+10,2,16.0,4035428.0,7.705863e+09
4,3.367588e+10,2,15.0,4865947.0,7.891111e+09


In [24]:
user_experience_df.rename(columns={'cluster': 'experience_cluster'}, inplace=True)
user_experience_df.head()

,MSISDN/Number,experience_cluster,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),experience_score
0,3.360100e+10,1,46.0,76.0,2895381.0,0.046970
1,3.360100e+10,1,31.0,99.0,9272453.0,0.100926
2,3.360100e+10,1,59.0,97.0,4150403.0,0.053453
3,3.360101e+10,1,84.0,248.0,2396.0,0.092104
4,3.360101e+10,2,119.0,56844.0,9738882.0,1.899030


In [27]:
user_satisfaction_df = pd.merge(user_engagement_df, user_experience_df, on='MSISDN/Number')
user_satisfaction_df['satisfaction_score'] = (
    user_satisfaction_df['experience_score'] + user_satisfaction_df['experience_score'])/2
user_satisfaction_df.head()

,MSISDN/Number,engagement_cluster,xDR Sessions,Dur. (ms),Total Data Volume (Bytes),experience_cluster,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),experience_score,satisfaction_score
0,3.362632e+10,0,18.0,8791927.0,7.971167e+09,2,709.0,121345.0,10558321.0,1.945114,1.945114
1,3.361489e+10,0,17.0,9966898.0,8.846226e+09,2,1049.0,426350.0,5895334.0,1.939431,1.939431
2,3.362578e+10,0,17.0,15466706.5,8.499621e+09,2,693.0,208539.0,695265.0,1.940463,1.940463
3,3.365973e+10,2,16.0,4035428.0,7.705863e+09,2,1078.0,285945.0,35736143.0,1.946593,1.946593
4,3.367588e+10,2,15.0,4865947.0,7.891111e+09,2,1062.0,203594.0,38657024.0,1.949862,1.949862


In [30]:

user_satisfaction_df = user_satisfaction_df[['MSISDN/Number', 'experience_score',
                        'experience_score', 'satisfaction_score']]
user_satisfaction_df.set_index('MSISDN/Number', inplace=True)
user_satisfaction_df.head()

,experience_score,experience_score,satisfaction_score
MSISDN/Number,,,
3.362632e+10,1.945114,1.945114,1.945114
3.361489e+10,1.939431,1.939431,1.939431
3.362578e+10,1.940463,1.940463,1.940463
3.365973e+10,1.946593,1.946593,1.946593
3.367588e+10,1.949862,1.949862,1.949862


In [31]:
top10_satisfied = user_satisfaction_df.sort_values('satisfaction_score', ascending=False).head(10)
top10_satisfied

,experience_score,experience_score,satisfaction_score
MSISDN/Number,,,
3.376054e+10,1.952180,1.952180,1.952180
3.367588e+10,1.949862,1.949862,1.949862
3.365973e+10,1.946593,1.946593,1.946593
3.362632e+10,1.945114,1.945114,1.945114
3.360452e+10,1.943099,1.943099,1.943099
3.362578e+10,1.940463,1.940463,1.940463
3.361489e+10,1.939431,1.939431,1.939431
3.360313e+10,1.936731,1.936731,1.936731
3.366716e+10,1.875070,1.875070,1.875070


### Linear Regression for satisfaction score prediction

In [33]:
# splitting training and testing data
X = user_satisfaction_df[['experience_score', 'experience_score']]
y = user_satisfaction_df[['satisfaction_score']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [34]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [35]:

y_pred = model.predict(X_test)

In [36]:
print(f'Intercept: {model.intercept_}')
print(f'Coefficients: { model.coef_}')
print(f"Mean squared error: {np.mean((y_pred - y_test.values) ** 2)}")

Intercept: [-1.33226763e-15]
Coefficients: [[0.25 0.25 0.25 0.25]]
Mean squared error: 0.0


### K-means Clusturing

In [38]:
# scale data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(user_satisfaction_df)
scaled_data

array([[ 0.4522495 ,  0.4522495 ,  0.4522495 ],
       [ 0.3606766 ,  0.3606766 ,  0.3606766 ],
       [ 0.3773041 ,  0.3773041 ,  0.3773041 ],
       [ 0.47609289,  0.47609289,  0.47609289],
       [ 0.52876857,  0.52876857,  0.52876857],
       [ 0.56612499,  0.56612499,  0.56612499],
       [-0.67649074, -0.67649074, -0.67649074],
       [ 0.31716648,  0.31716648,  0.31716648],
       [ 0.41977716,  0.41977716,  0.41977716],
       [-2.82166956, -2.82166956, -2.82166956]])

In [39]:
normalized_data = normalize(scaled_data)
normalized_data

array([[ 0.57735027,  0.57735027,  0.57735027],
       [ 0.57735027,  0.57735027,  0.57735027],
       [ 0.57735027,  0.57735027,  0.57735027],
       [ 0.57735027,  0.57735027,  0.57735027],
       [ 0.57735027,  0.57735027,  0.57735027],
       [ 0.57735027,  0.57735027,  0.57735027],
       [-0.57735027, -0.57735027, -0.57735027],
       [ 0.57735027,  0.57735027,  0.57735027],
       [ 0.57735027,  0.57735027,  0.57735027],
       [-0.57735027, -0.57735027, -0.57735027]])

In [40]:
kmeans = KMeans(n_clusters=2, random_state=1).fit(normalized_data)
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 1])

In [41]:

user_satisfaction_df.insert(0, 'satisfaction_cluster', kmeans.labels_)
user_satisfaction_df

,satisfaction_cluster,experience_score,experience_score,satisfaction_score
MSISDN/Number,,,,
3.362632e+10,0,1.945114,1.945114,1.945114
3.361489e+10,0,1.939431,1.939431,1.939431
3.362578e+10,0,1.940463,1.940463,1.940463
3.365973e+10,0,1.946593,1.946593,1.946593
3.367588e+10,0,1.949862,1.949862,1.949862
3.376054e+10,0,1.952180,1.952180,1.952180
3.366716e+10,1,1.875070,1.875070,1.875070
3.360313e+10,0,1.936731,1.936731,1.936731
3.360452e+10,0,1.943099,1.943099,1.943099


In [42]:

user_satisfaction_df['satisfaction_cluster'].value_counts()

0    8
1    2
Name: satisfaction_cluster, dtype: int64

In [48]:
# average satisfaction & experience score per cluster
#user_satisfaction_df.groupby('satisfaction_cluster').agg(
 #   {'satisfaction_score': 'mean', 'experience_score': 'mean'})

### Export dataframe to mysql

In [50]:
engine = create_engine('mysql+pymysql://root:@localhost/tellco')

In [ ]:
# save the clustering model
with open("../models/user_experience.pkl", "wb") as f:
    pickle.dump(kmeans, f)